In [1]:
import pandas as pd
df= pd.read_csv('Loan_ToPredict.csv')
df.head()

,Loan ID,Amount Requested,Amount Funded By Investors,Interest Rate,Loan Length,CREDIT Grade,Loan Title,Loan Purpose,Monthly PAYMENT,Total Amount Funded,...,Inquiries in the Last 6 Months,Accounts Now Delinquent,Delinquent Amount,Delinquencies (Last 2 yrs),Months Since Last Delinquency,Public Records On File,Months Since Last Record,Education,Employment Length,"Status (Fully Paid=1, Not Paid=0)"
0,1856772,10000,10000.00,10.16%,36 months,B1,Home improvement,home_improvement,323.43,10000,...,1.0,0.0,0.0,0.0,NaN,1.0,86.0,NaN,4 years,?
1,544915,4800,4800.00,13.61%,36 months,C2,My Loan,debt_consolidation,163.15,4800,...,0.0,0.0,0.0,0.0,39.0,0.0,NaN,NaN,< 1 year,?
2,2079551,10000,10000.00,12.12%,36 months,B3,consoliate,debt_consolidation,332.72,10000,...,2.0,0.0,0.0,1.0,8.0,0.0,NaN,NaN,n/a,?
3,2514828,5350,5350.00,15.80%,36 months,C3,Debt Consoldation,debt_consolidation,0.00,5350,...,1.0,0.0,0.0,1.0,12.0,0.0,NaN,NaN,1 year,?
4,396145,13000,2997.38,14.74%,36 months,D3,i always make my payment.,debt_consolidation,448.98,13000,...,3.0,0.0,0.0,0.0,NaN,0.0,NaN,Jefferson Community College-Louisville,3 years,?


In [2]:
#create a copy of the original data
df1 = df.copy()
#delete columns that are irrelevant
#del df1['Loan Title']
#del df1['Loan ID']
#del df1['City']

import numpy as np
#changing values of certain columns
df1['Interest Rate'] = df1['Interest Rate'].replace('%','',regex=True).astype('float')/100
df1['Debt-To-Income Ratio'] = df1['Debt-To-Income Ratio'].replace('%','',regex=True).astype('float')/100    
df1['Revolving Line Utilization'] = df1['Revolving Line Utilization'].replace('%','',regex=True).astype('float')/100



#normalizing the values of education
df1['Education'] = df1['Education'].replace(np.nan, 'Not Applicable', regex=True)
df1['Education'] = df1['Education'].replace(['n/a','none','None','NONE','undefined'], 'Not Applicable', regex=True)
searchForUni = ['uni','A&M','UCLA','V.C.U','unc','vcu','sdsu','san diego','Penn State','SUNY','Technology','UC','Dartmouth','FMU']
df1.loc[df1['Education'].str.contains('|'.join(searchForUni),case=False),'Education'] = 'University'
searchForCollege = ['college','ccny','NYCCT','coll']
df1.loc[df1['Education'].str.contains('|'.join(searchForCollege),case=False),'Education'] = 'College'
searchForVocational = ['tech','institute','Academy','Navy','vocational']
df1.loc[df1['Education'].str.contains('|'.join(searchForVocational),case=False),'Education'] = 'Vocational/Military School'

getTheRest = ['Not Applicable','University','College','Vocational/Military School']
df1.loc[~df1['Education'].isin(getTheRest),'Education'] = 'High School and Others' 




In [3]:
#df1['Education'].value_counts()

In [4]:
#df1['FICO Range'].unique()


#for column in df1:
#    print(column, " unique values!")
#    print(df1[column].unique())

In [5]:
#normalizing values of Employment Length
df1['Employment Length'].unique()
df1['Employment Length'] = df1['Employment Length'].replace('n/a','< 1 year',regex=True)

In [6]:
#changing values of FICO RANGE : NaN into mode
df1['FICO Range'] = df1['FICO Range'].replace(np.nan, '680-684', regex=True)
#df1['FICO Range'].value_counts()

In [7]:
df1['Months Since Last Record'].describe()
#change values of month since last record to mode


count    757.000000
mean      45.268164
std       47.576123
min        0.000000
25%        0.000000
50%       25.000000
75%       96.000000
max      129.000000
Name: Months Since Last Record, dtype: float64

In [8]:
def NaNgenerateRandom(df):
    mu = df.mean()
    std = df.std()
    a = df.values
    m = np.isnan(a) #no. of NANs
    a[m] = np.random.normal(mu,std,size=m.sum())
    a[m] = [int(round(x)) for x in a[m]]
    a[m] = [abs(x) for x in a[m]]
    return df

def NaNgenerateRandomPercent(df):
    mu = df.mean()
    std = df.std()
    a = df.values
    m = np.isnan(a) #no. of NANs
    a[m] = np.random.normal(mu,std,size=m.sum())
    a[m] = [abs(x) for x in a[m]]
    return df

In [9]:
df1['Months Since Last Record'] = NaNgenerateRandom(df1['Months Since Last Record'])

In [10]:
df1['Months Since Last Record'].describe()

count    8239.000000
mean       53.543148
std        38.667513
min         0.000000
25%        21.000000
50%        48.000000
75%        81.000000
max       202.000000
Name: Months Since Last Record, dtype: float64

In [11]:
#Randomize NaN with 0-5, with their respective probabilities
odds = [0.95405,0.04445,0.0014,0.00005,0.00005]
df1['Public Records On File'] = df1['Public Records On File'].replace(np.nan,np.random.choice(np.arange(0, 5), p=odds),regex=True)

In [12]:
#df1['Public Records On File'].value_counts()

In [13]:
df1['Months Since Last Delinquency'] = NaNgenerateRandom(df1['Months Since Last Delinquency'])
#df1['Months Since Last Delinquency'].describe()

In [14]:
#df1['Delinquencies (Last 2 yrs)'].value_counts()

In [15]:
df1['Delinquencies (Last 2 yrs)'] = NaNgenerateRandom(df1['Delinquencies (Last 2 yrs)'])

In [16]:
#df1['Delinquencies (Last 2 yrs)'].value_counts()

In [17]:
#df1['Revolving Line Utilization'].value_counts()

In [18]:
df1['Revolving Line Utilization'] = NaNgenerateRandomPercent(df1['Revolving Line Utilization'])
#df1['Revolving Line Utilization'].value_counts()

In [19]:
df1['Accounts Now Delinquent'].value_counts()
odds = [0.99,0.01]
df1['Accounts Now Delinquent'] = df1['Accounts Now Delinquent'].replace(np.nan,np.random.choice(np.arange(0, 2), p=odds),regex=True)

In [20]:
df1['Revolving CREDIT Balance'] = NaNgenerateRandom(df1['Revolving CREDIT Balance'])

In [21]:
#df1['Inquiries in the Last 6 Months'].value_counts()
df1['Inquiries in the Last 6 Months'] = NaNgenerateRandom(df1['Inquiries in the Last 6 Months'])
#df1['Inquiries in the Last 6 Months'].value_counts()

In [22]:
df1['Total CREDIT Lines'] = NaNgenerateRandom(df1['Total CREDIT Lines'])
#df1['Total CREDIT Lines'].value_counts()

In [23]:
df1['Open CREDIT Lines'] = NaNgenerateRandom(df1['Open CREDIT Lines'])

In [24]:
df1['Earliest CREDIT Line'] = df1['Earliest CREDIT Line'].replace(np.nan,'12/8/95',regex=True)

In [25]:

df1.loc[(df1['Monthly Income']< 0,'Monthly Income')] = 0

In [26]:
df1['Debt-To-Income Ratio'] = NaNgenerateRandomPercent(df1['Debt-To-Income Ratio'])
df1['Interest Rate'] = NaNgenerateRandomPercent(df1['Interest Rate'])

In [27]:


df1.loc[(df1['Total Amount Funded']< 0, 'Total Amount Funded')] = 0

In [28]:


df1.loc[(df1['Monthly PAYMENT']< 0, 'Monthly PAYMENT')] = 0

In [29]:
df1['Loan Length'].value_counts()

36 months    6845
60 months    1394
Name: Loan Length, dtype: int64

In [30]:
odds = ['36 months'] * 16725 + ['60 months'] * 3275
df1['Accounts Now Delinquent'] = df1['Accounts Now Delinquent'].replace(np.nan,np.random.choice(odds),regex=True)

In [31]:
df1.loc[(df1['Amount Requested']< 0, 'Amount Requested')] = 0
        
df1.loc[(df1['Amount Funded By Investors']< 0, 'Amount Funded By Investors')] = 0

In [69]:
df1.head()

,Amount Requested,Amount Funded By Investors,Interest Rate,Loan Length,CREDIT Grade,Loan Purpose,Monthly PAYMENT,Total Amount Funded,Debt-To-Income Ratio,State,...,Inquiries in the Last 6 Months,Accounts Now Delinquent,Delinquent Amount,Delinquencies (Last 2 yrs),Months Since Last Delinquency,Public Records On File,Months Since Last Record,Education,Employment Length,"Status (Fully Paid=1, Not Paid=0)"
0,10000,10000.00,0.1016,36 months,B1,home_improvement,323.43,10000,0.1745,TX,...,1.0,0.0,0.0,0.0,39.0,1.0,86.0,Not Applicable,4 years,?
1,4800,4800.00,0.1361,36 months,C2,debt_consolidation,163.15,4800,0.1579,OH,...,0.0,0.0,0.0,0.0,39.0,0.0,30.0,Not Applicable,< 1 year,?
2,10000,10000.00,0.1212,36 months,B3,debt_consolidation,332.72,10000,0.1484,CO,...,2.0,0.0,0.0,1.0,8.0,0.0,130.0,Not Applicable,< 1 year,?
3,5350,5350.00,0.1580,36 months,C3,debt_consolidation,0.00,5350,0.2136,TX,...,1.0,0.0,0.0,1.0,12.0,0.0,83.0,Not Applicable,1 year,?
4,13000,2997.38,0.1474,36 months,D3,debt_consolidation,448.98,13000,0.1639,KY,...,3.0,0.0,0.0,0.0,48.0,0.0,15.0,University,3 years,?


In [34]:
df1.to_csv('cleanTest.csv', encoding='utf-8',index= False)